In [2]:
import matplotlib.pyplot as plt
from nptdms import TdmsFile
import numpy as np
import pandas as pd

In [3]:
with TdmsFile.open(r'D:\Desktop\BasicProject\my_file.tdms') as tdms_file:
    for group in tdms_file.groups():      # TdmsFile可以按组名索引来访问TDMS文件中的组，使用groups()方法直接访问所有组
        group_name = group.name
        print(group_name)
    for channel in group.channels():      # TdmsGroup 可以通过通道名称来索引来访问这个组中的一个通道，使用 channels()方法直接访问所有通道
        channel_name = channel.name
        print(channel_name)
    channel = tdms_file['group_1']['channel_1']  # 根据索引读取通道
    all_channel_data = channel[:]                  # 将此通道中所有的数据作为numpy数组获取
    num = np.array(all_channel_data)
    print(num)



FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Desktop\\BasicProject\\my_file.tdms'

In [21]:
tdms_file = TdmsFile.read(r"C:\Users\EDY\Downloads\exampleMeasurements.tdms")

In [27]:
type(tdms_file)

nptdms.tdms.TdmsFile

In [31]:
all_groups = tdms_file.groups()
for group in all_groups:
    print(group.name)
print(all_groups)

EHM
[<TdmsGroup with path /'EHM'>]


In [32]:
all_group_channels = group.channels()
for channel in all_group_channels:
    print(channel)

<TdmsChannel with path /'EHM'/'VaGroundV'>
<TdmsChannel with path /'EHM'/'VbGroundV'>
<TdmsChannel with path /'EHM'/'VcGroundV'>
<TdmsChannel with path /'EHM'/'VaVbV'>
<TdmsChannel with path /'EHM'/'VbVcV'>
<TdmsChannel with path /'EHM'/'VcVaV'>
<TdmsChannel with path /'EHM'/'CurrentA'>
<TdmsChannel with path /'EHM'/'CurrentB'>
<TdmsChannel with path /'EHM'/'CurrentC'>
<TdmsChannel with path /'EHM'/'SensoPowerV'>
<TdmsChannel with path /'EHM'/'TorqueNm'>
<TdmsChannel with path /'EHM'/'LoadN'>
<TdmsChannel with path /'EHM'/'TemperatureC'>
<TdmsChannel with path /'EHM'/'HallSensor'>
<TdmsChannel with path /'EHM'/'SoundPressurePa'>
<TdmsChannel with path /'EHM'/'AccelX'>
<TdmsChannel with path /'EHM'/'AccelY'>
<TdmsChannel with path /'EHM'/'AccelZ'>


In [33]:
all_data = tdms_file['EHM']['AccelX'][:]
all_data

array([ 0.00433087, -0.00256777, -0.00536799, ..., -0.00800729,
        0.00939131,  0.01466393])

In [35]:
all_data2 = tdms_file['EHM']['AccelY'][:]
all_data2

array([ 0.00436544, -0.01236439, -0.017277  , ..., -0.00970602,
        0.00852346,  0.01237035])

In [17]:
channel = tdms_file[group_name][channel_name]  # 根据索引读取通道
all_channel_data = channel[:]                  # 将此通道中所有的数据作为numpy数组获取
num = np.array(all_channel_data)
num                              # 维度查看


RuntimeError: Cannot read data after the underlying TDMS reader is closed

In [13]:
from nptdms import TdmsWriter, RootObject, GroupObject, ChannelObject

root_object = RootObject(properties={
    "prop1": "foo",
    "prop2": 3,
})
group_object = GroupObject("group_1", properties={
    "prop1": 1.2345,
    "prop2": False,
})
data = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
channel_object = ChannelObject("group_1", "channel_1", data, properties={})

with TdmsWriter(r"D:\Desktop\BasicProject\my_file.tdms") as tdms_writer:
    # Write first segment
    tdms_writer.write_segment([
        root_object,
        group_object,
        channel_object])
    # Write another segment with more data for the same channel
    more_data = np.array([6.0, 7.0, 8.0, 9.0, 10.0])
    channel_object = ChannelObject("group_1", "channel_1", more_data, properties={})
    tdms_writer.write_segment([channel_object])

In [1]:
import math

In [3]:
x = 0
y = 1
math.atan2(y, x)

1.5707963267948966

In [103]:
import json
import pprint
import numpy as np
import numpy.linalg as la
with open (r'D:\Desktop\Project_file\张子千\audi\a2d2-preview\cams_lidars.json', 'r') as f:
    config = json.load(f)
EPSILON = 1.0e-10 # norm should not be small
def get_axes_of_a_view(view):
    x_axis = view['x-axis']
    y_axis = view['y-axis']

    x_axis_norm = la.norm(x_axis)
    y_axis_norm = la.norm(y_axis)

    if (x_axis_norm < EPSILON or y_axis_norm < EPSILON):
        raise ValueError("Norm of input vector(s) too small.")

    # normalize the axes
    x_axis = x_axis / x_axis_norm
    y_axis = y_axis / y_axis_norm

    # make a new y-axis which lies in the original x-y plane, but is orthogonal to x-axis
    y_axis = y_axis - x_axis * np.dot(y_axis, x_axis)

    # create orthogonal z-axis
    z_axis = np.cross(x_axis, y_axis)

    # calculate and check y-axis and z-axis norms
    y_axis_norm = la.norm(y_axis)
    z_axis_norm = la.norm(z_axis)

    if (y_axis_norm < EPSILON) or (z_axis_norm < EPSILON):
        raise ValueError("Norm of view axis vector(s) too small.")

    # make x/y/z-axes orthonormal
    y_axis = y_axis / y_axis_norm
    z_axis = z_axis / z_axis_norm

    return x_axis, y_axis, z_axis
def get_origin_of_a_view(view):
    return view['origin']
def get_transform_to_global(view):
    # get axes
    x_axis, y_axis, z_axis = get_axes_of_a_view(view)

    # get origin
    origin = get_origin_of_a_view(view)
    transform_to_global = np.eye(4)

    # rotation
    transform_to_global[0:3, 0] = x_axis
    transform_to_global[0:3, 1] = y_axis
    transform_to_global[0:3, 2] = z_axis

    # origin
    transform_to_global[0:3, 3] = origin

    return transform_to_global

views = ['front_center', 'front_left', 'front_right', 'rear_left', 'rear_right']
# for view in :
#     lidar = config['lidars'][view]['view']
#     ext = get_transform_to_global(lidar)
lidar_1 = config['lidars']['front_center']['view']
lidar_2 = config['lidars']['front_left']['view']
lidar_3 = config['lidars']['front_right']['view']
lidar_4 = config['lidars']['rear_left']['view']
lidar_5 = config['lidars']['rear_right']['view']
# lidar_6 = config['lidars']['front_center']['view']

ext1 = get_transform_to_global(lidar_1)
ext2 = get_transform_to_global(lidar_2)
ext3 = get_transform_to_global(lidar_3)
ext4 = get_transform_to_global(lidar_4)
ext5 = get_transform_to_global(lidar_5)
# ext6 = get_transform_to_global(lidar_6)

In [110]:
f1 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\lidar\cam_front_center\20180810150607_lidar_frontcenter_000000083.npz"
f2 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\lidar\cam_front_left\20180810150607_lidar_frontleft_000000083.npz"
f3 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\lidar\cam_front_right\20180810150607_lidar_frontright_000000083.npz"
f6 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\lidar\cam_rear_center\20180810150607_lidar_rearcenter_000000060.npz"
f4 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\lidar\cam_side_left\20180810150607_lidar_sideleft_000000060.npz"
f5 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\lidar\cam_side_right\20180810150607_lidar_sideright_000000060.npz"

In [113]:
def parse_points(datas, ext):
    # i = datas['pcloud_attr.reflectance']
    point = datas['pcloud_points']
    i = np.ones(len(point))
    points = np.hstack((point, i.reshape((-1,1)))) @ ext
    return points
def write_pcd(points):
    with open(rf"D:\Desktop\Project_file\张子千\audi\新建文件夹\merge_use_lidar.pcd", 'w', encoding='ascii') as pcd_file:
        point_num = points.shape[0]
        heads = [
            '# .PCD v0.7 - Point Cloud Data file format',
            'VERSION 0.7',
            'FIELDS x y z i',
            'SIZE 4 4 4 4',
            'TYPE F F F F',
            'COUNT 1 1 1 1',
            f'WIDTH {point_num}',
            'HEIGHT 1',
            'VIEWPOINT 0 0 0 1 0 0 0',
            f'POINTS {point_num}',
            'DATA ascii'
        ]
        pcd_file.write('\n'.join(heads))
        for i in range(point_num):
            string_point = '\n' + str(points[i, 0]) + ' ' + str(points[i, 1]) + ' ' + str(points[i, 2]) + ' ' + str(points[i, 3])
            pcd_file.write(string_point)

In [114]:
files = [f1, f2, f3, f4, f5]
points = np.array([[0, 0, 0, 1]])
for i in range(5):
    datas = np.load(files[i])
    lidar = config['lidars'][views[i]]['view']
    ext = get_transform_to_global(lidar)
    point = parse_points(datas, ext)
    points = np.around(np.vstack((points, point)), 3)
    write_pcd(points)



In [85]:
np.set_printoptions(precision=3, suppress=True)
ext1

array([[-0.008,  0.981,  0.195,  1.718],
       [-1.   , -0.006, -0.01 ,  0.   ],
       [-0.009, -0.195,  0.981,  1.12 ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

array([[ 18.36098678,  22.27193186,  -1.17964227,  34.        ],
       [ 23.61544058,  28.63543877,   1.07865641,  57.        ],
       [ 18.47636053,  22.24693162,  -1.16846125,  34.        ],
       ...,
       [ 54.14718759, -58.2765401 ,  11.84631328,  63.        ],
       [ 54.04730885, -58.53217118,  11.98414589,  63.        ],
       [ 53.89733437, -59.48843391,  12.44430544,  63.        ]])

In [102]:
def write_pcd2(num, points):
    with open(rf"D:\Desktop\Project_file\张子千\audi\新建文件夹\t{num}.pcd", 'w', encoding='ascii') as pcd_file:
        point_num = points.shape[0]
        heads = [
            '# .PCD v0.7 - Point Cloud Data file format',
            'VERSION 0.7',
            'FIELDS x y z i',
            'SIZE 4 4 4 4',
            'TYPE F F F F',
            'COUNT 1 1 1 1',
            f'WIDTH {point_num}',
            'HEIGHT 1',
            'VIEWPOINT 0 0 0 1 0 0 0',
            f'POINTS {point_num}',
            'DATA ascii'
        ]
        pcd_file.write('\n'.join(heads))
        for i in range(point_num):
            string_point = '\n' + str(points[i, 0]) + ' ' + str(points[i, 1]) + ' ' + str(points[i, 2]) + ' ' + str(points[i, 3])
            pcd_file.write(string_point)

In [89]:
files = [f1, f2, f3, f4, f5]
for i in range(5):
    datas = np.load(files[i])
    lidar = config['lidars'][views[i]]['view']
    ext = get_transform_to_global(lidar)
    points = np.around(parse_points(datas, ext), 3)
    write_pcd2(points)

In [111]:
c1 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\camera\cam_front_center\20180810150607_camera_frontcenter_000000083.json"
c2 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\camera\cam_front_left\20180810150607_camera_frontleft_000000083.json"
c3 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\camera\cam_front_right\20180810150607_camera_frontright_000000083.json"
c4 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\camera\cam_side_left\20180810150607_camera_sideleft_000000060.json"
c5 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\camera\cam_side_right\20180810150607_camera_sideright_000000060.json"
c6 = r"D:\Desktop\Project_file\张子千\audi\a2d2-preview\camera_lidar\20180810_150607\camera\cam_rear_center\20180810150607_camera_rearcenter_000000060.json"
cs = [c1, c2, c3, c4, c5, c6]
def parse_ext(file):
    with open(file, 'r') as tf:
        cfg = json.load(tf)
        ext_t = get_transform_to_global(cfg['pcld_view'])
        return ext_t

In [112]:
files = [f1, f2, f3, f4, f5, f6]
points = np.array([[0, 0, 0, 1]])
for i in range(6):
    datas = np.load(files[i])
    ext = parse_ext(cs[i])
    point = parse_points(datas, ext)
    points = np.around(np.vstack((points, point)), 3)
    write_pcd(points)

In [109]:
data = {
    "3d_img0":{
    "camera_internal": {
                    "fx": 1844.1774422790927,
                    "fy": 1841.5212239377258,
                    "cx": 964.42990523863864,
                    "cy": 679.5331911948183
                },
    "camera_external": ext_t.flatten().tolist()
    }}
with open(r"D:\Desktop\Project_file\张子千\audi\新建文件夹\config.json", 'w') as cf:
    json.dump(data, cf)